In [3]:
# Basic
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import random
from tqdm import tqdm
import time
import re

# For BERTopic
from bertopic import BERTopic
import openai

# For saving and loading model
import pickle

/dlabscratch1/bdoan/.conda/envs/qan/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
topics_df = pd.read_csv('bert/bert-results/topics_all.csv')
topics_df

,Unnamed: 0,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,0,-1,128667,-1_covid_vaccine_daughter_son,"['covid', 'vaccine', 'daughter', 'son', 'vacci...","['pandemic', 'grandma', 'daughter', 'abuse', '...","['covid', 'vaccine', 'daughter', 'vaccines', '...","[""Now she outright refuses to get boosted, bla..."
1,1,0,3658,0_qanon_qanon conspiracy_qanon conspiracies_qa...,"['qanon', 'qanon conspiracy', 'qanon conspirac...","['qanon stuff', 'qanon', 'qanon bullshit', 'qa...","['qanon', 'qanon conspiracy', 'qanon conspirac...","['Well I oficially lost my father to QAnon.', ..."
2,2,1,3225,1_mom back_want mom_mom mom_mum,"['mom back', 'want mom', 'mom mom', 'mum', 'lo...","['mom back', 'mom want', 'mom wanted', 'mom go...","['mom back', 'want mom', 'love mom', 'lost mom...","['I just want my mom back.', 'I just want my m..."
3,3,2,2977,2_conspiracy theories_theories_conspiracies_co...,"['conspiracy theories', 'theories', 'conspirac...","['conspiracy theories', 'conspiracy theory', '...","['conspiracy theories', 'conspiracy theorists'...","['I was openminded at first, but the conspirac..."
4,4,3,2615,3_dad dad_love dad_dad always_like dad,"['dad dad', 'love dad', 'dad always', 'like da...","['love dad', 'dad always', 'talk dad', 'talkin...","['love dad', 'step dad', 'dad said', 'dad call...",['I love my dad but this is tough to say the l...
...,...,...,...,...,...,...,...,...
457,457,456,50,456_letter_letters_writing_mail,"['letter', 'letters', 'writing', 'mail', 'addr...","['letter', 'letters', 'mail', 'writing', 'writ...","['letter', 'mail', 'addressed', 'grandmother',...",['&#x200B;\n\nUPDATE: I decided not to share t...
458,458,457,50,457_requiring_vaccinations_college_university,"['requiring', 'vaccinations', 'college', 'univ...","['get vaccinated', 'vaccination', 'get vaccine...","['vaccinations', 'mandate', 'require', 'mandat...","[""3. the school is very strict about the vacci..."
459,459,458,50,458_sister got_sibling_sisters_verbally,"['sister got', 'sibling', 'sisters', 'verbally...","['brother lives', 'sibling', 'emotionally abus...","['sibling', 'sisters', 'apologized', 'abused',...",['We decided to try the conversation one more ...
460,460,459,50,459_parler_account_facebook_gab,"['parler', 'account', 'facebook', 'gab', 'join...","['parler', 'twitter', 'treason', 'shut', 'glee...","['parler', 'facebook', 'gab', 'twitter', 'rece...","['They are all on parler as well.', 'ColonelTP..."


In [12]:
topics_df = topics_df.drop(['Unnamed: 0', 'KeyBERT', 'MMR'], axis=1)
topics_df = topics_df.drop(0, axis=0)
topics_df

,Topic,Count,Name,Representation,Representative_Docs
1,0,3658,0_qanon_qanon conspiracy_qanon conspiracies_qa...,"['qanon', 'qanon conspiracy', 'qanon conspirac...","['Well I oficially lost my father to QAnon.', ..."
2,1,3225,1_mom back_want mom_mom mom_mum,"['mom back', 'want mom', 'mom mom', 'mum', 'lo...","['I just want my mom back.', 'I just want my m..."
3,2,2977,2_conspiracy theories_theories_conspiracies_co...,"['conspiracy theories', 'theories', 'conspirac...","['I was openminded at first, but the conspirac..."
4,3,2615,3_dad dad_love dad_dad always_like dad,"['dad dad', 'love dad', 'dad always', 'like da...",['I love my dad but this is tough to say the l...
5,4,2572,4_text_screamed_mad_hung,"['text', 'screamed', 'mad', 'hung', 'argue', '...",['I was hyperventilating and crying uncontroll...
...,...,...,...,...,...
457,456,50,456_letter_letters_writing_mail,"['letter', 'letters', 'writing', 'mail', 'addr...",['&#x200B;\n\nUPDATE: I decided not to share t...
458,457,50,457_requiring_vaccinations_college_university,"['requiring', 'vaccinations', 'college', 'univ...","[""3. the school is very strict about the vacci..."
459,458,50,458_sister got_sibling_sisters_verbally,"['sister got', 'sibling', 'sisters', 'verbally...",['We decided to try the conversation one more ...
460,459,50,459_parler_account_facebook_gab,"['parler', 'account', 'facebook', 'gab', 'join...","['They are all on parler as well.', 'ColonelTP..."


### 1. Determine relevance of topics

Manually-annotated few-shot examples

In [13]:
few_shots = """
Topic representation: ['know alone', 'feel alone', 'comforting', 'nice know', 'comfort', 'alone know', 'lonely', 'nice', 'good know', 'knowing alone']
Representative docs: ["It's just so comforting to know I am not alone.", "it's so comforting to know I'm not alone.", "It's comforting to know I'm not alone in this."]
Label: No
Reason: These reflect the narrator’s emotional experience in posting, not the OON’s radicalization pathway.

---
Topic representation: ['blocked', 'block', 'blocking', 'social media', 'unfriended', 'number', 'facebook', 'deleted', 'account', 'fb']
Representative docs: ["And then I blocked her.", "I didn't know because I blocked her on everything.", "I just blocked her on my phone now."]
Label: Yes
Reason: This reflects post-radicalization consequences — how the narrator responded to the OON's behavior (Stage 3: After).

---
Topic representation: ['qanon', 'qanon stuff', 'qanon conspiracy', 'qanon conspiracies', 'qanon rabbit hole', 'qanon rabbit', 'theories', 'conspiracy theories', 'believer', 'conspiracies']
Representative docs: ["In her case, she doesn’t partake of QAnon.", "I once asked her if she knew what QAnon was and she said no.", "One question I had was: how did she find out about QAnon?"]
Label: No
Reason: These are superficial mentions of QAnon but lack insight into the OON’s specific beliefs or behavior.

---
Topic representation: ['pandemic', 'pandemic started', 'since pandemic', 'plandemic', 'beginning pandemic', 'pandemic hit', 'ever since', 'beginning', 'went deep', 'start pandemic']
Representative docs: ["And she calls the pandemic a 'pland-emic'.", "It seemed that as the pandemic wore on, she became more and more cemented in her views.", "But the pandemic changed her."]
Label: Yes
Reason: This clearly indicates a triggering event contributing to radicalization (Stage 2: Trigger).

---
Topic representation: ['red', 'texas', 'florida', 'area', 'rural', 'town', 'county', 'city', 'states', 'blue']
Representative docs: ["We live in a red state.", "We're also in a red state.", "Still, he is deep in a red state.", "We are from a very small rural red town.", "I live in a very red area."]
Label: Yes
Reason: This offers background context about the OON’s political environment, useful for understanding predispositions (Stage 1: Before).

---
Topic representation: ['know know', 'know even', 'even know']
Representative docs: ["I don’t know what to do.", "I don’t know what to do.", "I don’t even know what to do."]
Label: No
Reason: The sentences reflect the narrator’s distress but do not give meaningful info about the OON.

"""

In [14]:
topics_df["Relevance"] = ""
topics_df["Relevance_Reason"] = ""

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Get the API key
api_key = os.getenv("DLAB_API_KEY")
if not api_key:
    raise ValueError("DLAB_API_KEY not found in .env file or environment variables")

In [15]:
client = openai.OpenAI(api_key=api_key)

In [16]:
test_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"},
    ],
    temperature=0.3,
)

print(test_response.choices[0].message.content.strip())

Hello! How can I assist you today?


In [26]:
representation = ['gates', 'bill gates', 'bill', '5g', 'microchips', 'microchip', 'dna', 'vaccines', 'kill', 'population']
representative_docs = ['* Bill Gates - eugenics, decreasing the population, and vaccines\n* Globalism/NWO - George Soros, the Rockefellers, Jeff Bezos, Bill Gates, Klaus Schwabb, etc.', "And it was Bill Gates' company.", 'And of course the 5G, Covid, and Bill Gates conspiracies make sense to them.', 'Bill Gates is trying to implant microchips.', "I honestly still don't understand the Bill Gates thing."]

prompt = f"""
You are a qualitative coding assistant helping analyze radicalization pathways from r/QAnonCasualties posts.

Determine whether the following topic is relevant to understanding the Object of Narration’s (OON’s) radicalization pathway.
Only say Yes if it contributes meaningful information to one of these stages: 
1. Before radicalization (e.g., traits, background)
2. Trigger (e.g., pandemic, influencers)
3. After radicalization (e.g., beliefs, behaviors, family impact)

Give Label: Yes/No and Reason.

{few_shots}
---
Topic representation: {representation}
Representative docs: {representative_docs}
Label:"""
    
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content.strip()
    print(answer)
except Exception as e:
    print(f"Error: {e}")

Label: Yes  
Reason: This topic discusses specific conspiracy theories related to Bill Gates and vaccines, which are relevant to the OON's beliefs and behaviors after radicalization (Stage 3: After).


In [28]:
lines = answer.strip().split('\n')
label = lines[0].split('Label:')[1].strip()
reason = lines[1].split('Reason:')[1].strip()

print("Label:", label)
print("Reason:", reason)

Label: Yes
Reason: This topic discusses specific conspiracy theories related to Bill Gates and vaccines, which are relevant to the OON's beliefs and behaviors after radicalization (Stage 3: After).


In [36]:
def check_topic_relevance(representation, representative_docs):
    prompt = f"""
You are a qualitative coding assistant helping analyze radicalization pathways from r/QAnonCasualties posts.

Determine whether the following topic is relevant to understanding the Object of Narration’s (OON’s) radicalization pathway.
Only say Yes if it contributes meaningful information to one of these stages: 
1. Before radicalization (e.g., traits, background)
2. Trigger (e.g., pandemic, influencers)
3. After radicalization (e.g., beliefs, behaviors, family impact)

Give Label: Yes/No and Reason.

{few_shots}
---
Topic representation: {representation}
Representative docs: {representative_docs}
"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            temperature=0.2,
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return "Label: ERROR\nReason: API call failed"

# === Run the loop ===
output = []
for idx, row in tqdm(topics_df.iterrows(), total=len(topics_df)):
    rep = row['Representation']
    rep_docs = row['Representative_Docs']

    result = check_topic_relevance(rep, rep_docs)

    lines = result.strip().split('\n')
    topics_df.at[idx, "Relevance"] = lines[0].split('Label:')[1].strip()
    topics_df.at[idx, "Relevance_Reason"] = lines[1].split('Reason:')[1].strip()
    time.sleep(1.5)  # avoid rate limit

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 461/461 [19:53<00:00,  2.59s/it]


In [37]:
topics_df

,Topic,Count,Name,Representation,Representative_Docs,Relevance,Relevance_Reason
1,0,3658,0_qanon_qanon conspiracy_qanon conspiracies_qa...,"['qanon', 'qanon conspiracy', 'qanon conspirac...","['Well I oficially lost my father to QAnon.', ...",No,"While there are mentions of QAnon, the focus i..."
2,1,3225,1_mom back_want mom_mom mom_mum,"['mom back', 'want mom', 'mom mom', 'mum', 'lo...","['I just want my mom back.', 'I just want my m...",No,The expressions of longing for the mother indi...
3,2,2977,2_conspiracy theories_theories_conspiracies_co...,"['conspiracy theories', 'theories', 'conspirac...","['I was openminded at first, but the conspirac...",Yes,This topic provides insight into the OON's bel...
4,3,2615,3_dad dad_love dad_dad always_like dad,"['dad dad', 'love dad', 'dad always', 'like da...",['I love my dad but this is tough to say the l...,Yes,These statements reflect the emotional connect...
5,4,2572,4_text_screamed_mad_hung,"['text', 'screamed', 'mad', 'hung', 'argue', '...",['I was hyperventilating and crying uncontroll...,Yes,This reflects the emotional and communicative ...
...,...,...,...,...,...,...,...
457,456,50,456_letter_letters_writing_mail,"['letter', 'letters', 'writing', 'mail', 'addr...",['&#x200B;\n\nUPDATE: I decided not to share t...,No,The topic focuses on the act of writing letter...
458,457,50,457_requiring_vaccinations_college_university,"['requiring', 'vaccinations', 'college', 'univ...","[""3. the school is very strict about the vacci...",Yes,This topic discusses vaccination mandates in a...
459,458,50,458_sister got_sibling_sisters_verbally,"['sister got', 'sibling', 'sisters', 'verbally...",['We decided to try the conversation one more ...,Yes,This topic discusses family dynamics and the i...
460,459,50,459_parler_account_facebook_gab,"['parler', 'account', 'facebook', 'gab', 'join...","['They are all on parler as well.', 'ColonelTP...",Yes,This indicates the OON's engagement with alter...


In [39]:
topics_df.to_csv('llm_relevance.csv')

In [40]:
manual_df = pd.read_excel('data/topics_merge_test (1).xlsx')
manual_df

,Unnamed: 0,Topic,Count,Name,Representation,Representative_Docs,GPT Label,Better Label?,"Category (0 - Background, 1 - Oon before, 2 - Trigger, 3 - Oon after)",Remarks
0,0,1,3225,1_mom back_want mom_mom mom_mum,"['mom back', 'want mom', 'mom mom', 'mum', 'lo...","['I just want my mom back.', 'I want my mom ba...",Desire for Reconnection with Mother,NaN,NaN,Narr's needs
1,1,3,2615,3_dad dad_love dad_dad always_like dad,"['dad dad', 'love dad', 'dad always', 'like da...","['My dad was always easier.', 'My dad and I ha...",Strained Father-Child Relationships Due to QAnon,NaN,3-Oon after-Social,NaN
2,2,4,2572,4_text_screamed_mad_hung,"['text', 'screamed', 'mad', 'hung', 'argue', '...",['He asks a lot of questions and I just try to...,Family Conflict Over QAnon Beliefs,Q's Hostile Behavior towards others,3-Oon after-Behavior/action,NaN
3,3,5,1981,5_text_argue_spoken_want talk,"['text', 'argue', 'spoken', 'want talk', 'hung...","[""I haven't spoken to her since then and it's ...",Family Communication Breakdown Due to QAnon Be...,NaN,3-Oon after-Behavior/action,NaN
4,4,13,1121,13_doctor_doctors_cancer_hospital,"['doctor', 'doctors', 'cancer', 'hospital', 'm...","['(Give her cancer or something).', 'Not to me...",Family Impact of Medical Misinformation in QAn...,Distrust in doctors?,3-Oon after-Behavior/action,NaN
...,...,...,...,...,...,...,...,...,...,...
257,274,"211,260,303",300,NaN,"['clean', 'completely cut', 'cut completely', ...","[""And while they aren't saints in all this, I ...",Family Estrangement Due to QAnon Beliefs,NaN,3-Oon after-Social,NaN
258,276,"24,311,350",906,NaN,"['10 years', '20 years', 'best friend', 'best ...","['She was always filthy, and she was my best f...",Strained Friendships Due to QAnon Involvement,Oon and narrator relationship - best friend,0-Background-Relationship with narr,NaN
259,277,"23,97,263",1142,NaN,"['anything political', 'apolitical', 'boundari...","['We agree, again - no more politics.', 'New f...",Family Dynamics and Politics Boundaries,NaN,3-Oon after-Social,NaN
260,278,"11,25,61",2386,NaN,"['2016', 'conservative', 'democrat', 'fraud', ...","['2016 I found out he voted for Trump.', 'He i...",Family Dynamics and Political Differences,NaN,0-Background-Ideology,Is it better to distangle this? (But it's also...


In [68]:
# 1. Normalize manual_df to get a flat set of topic indices you considered relevant

manual_indices = set()
for item in manual_df["Topic"]:
    idxs = str(item).split(',')
    idxs = [int(i) for i in idxs]
    manual_indices.update(idxs)

# 2. Get LLM-kept topic indices
llm_indices = set(topics_df[topics_df["Relevance"] == "Yes"]['Topic'])

# 3. Find disagreements
manual_but_not_llm = manual_indices - llm_indices
llm_but_not_manual = llm_indices - manual_indices

# 4. Optional: Output the relevant rows for inspection
manual_but_not_llm_df = topics_df[topics_df['Topic'].isin(manual_but_not_llm)]
llm_but_not_manual_df = topics_df[topics_df['Topic'].isin(llm_but_not_manual)]

print("Topics I kept but LLM did not:")
display(manual_but_not_llm_df[['Topic', 'Representation', 'Representative_Docs', 'Relevance', 'Relevance_Reason']])

print("Topics LLM kept but I did not:")
display(llm_but_not_manual_df[['Topic', 'Representation', 'Representative_Docs', 'Relevance', 'Relevance_Reason']])

Topics I kept but LLM did not:


,Topic,Representation,Representative_Docs,Relevance,Relevance_Reason
2,1,"['mom back', 'want mom', 'mom mom', 'mum', 'love mom', 'relationship mom', 'miss', 'mom always', 'mom want', 'like mom']","['I just want my mom back.', 'I just want my mom back.', 'Seriously I want my mom back.']",No,"The expressions of longing for the mother indicate emotional distress but do not provide relevant information about the OON's radicalization pathway or any specific traits, triggers, or post-radicalization behaviors."
14,13,"['doctor', 'doctors', 'cancer', 'hospital', 'medical', 'meds', 'medicine', 'medication', 'cure', 'treatment']","['I urged her to get to her doctor.', 'You cannot just look at her and say she’s not a doctor.', ""She is the hospital right now, and I don't how long she's had the cancer yet.""]",No,"The topic focuses on medical issues and interactions with healthcare professionals, which do not provide meaningful insights into the OON's radicalization pathway."
18,17,"['biden', 'joe biden', 'joe', 'president', 'hunter', 'hunter biden', 'president biden', 'sworn', 'voted', 'harris']","['""Biden didn\'t take the oath.', 'That Biden might be one and all that.', 'You support Biden and *he* gave massages!']",No,The mentions of Biden and related political figures do not provide specific insights into the OON's beliefs or behaviors related to radicalization. They are more about political opinions rather than contributing to understanding the radicalization pathway.
45,44,"['conversations', 'spoken', 'spoken since', 'communication', 'end conversation', 'want talk', 'talk phone', 'talk anything', 'conversation went', 'small talk']","['Any conversations we have can be kept confidential for now.', 'We do not talk often so it never comes up in our conversations.', 'Our conversations were only about this.']",No,"The topic focuses on the nature of conversations and communication patterns, which do not provide meaningful insights into the OON's radicalization pathway or any specific stages of radicalization."
56,55,"['therapy', 'therapist', 'counseling', 'therapists', 'couples', 'counselor', 'appointment', 'session', 'sessions', 'insurance']","['I’m working on it with therapy.', ""I've been through years of therapy."", 'I have recently been going to therapy to deal with these issues.']",No,The topic focuses on the narrator's personal experiences with therapy and does not provide meaningful information about the OON’s radicalization pathway.
60,59,"['miss', 'love much', 'still love', 'feel bad', 'senses', 'loves', 'want relationship', 'person used', 'come back', 'get back']","['I just miss him so very, very much!', 'I miss him so much.', 'I just miss him and wish it wasn’t like this.']",No,These expressions of longing and emotional distress do not provide meaningful information about the OON’s radicalization pathway or any relevant stages of radicalization.
61,60,"['cried', 'cry', 'crying', 'tears', 'scream', 'sobbing', 'laugh', 'void', 'eyes', 'broke']","['Maybe I cried all I could that day in the barn.', 'I cried, I was angry, I had so much sorrow, and I grieved.', 'I have cried so many tears and am on anti depressants to help me out.']",No,These expressions of emotional distress and grief pertain to the narrator's personal feelings rather than providing meaningful insights into the OON’s radicalization pathway.
70,69,"['boundaries', 'bring back', 'like lost', 'want back', 'breaks heart', 'feel like lost', 'want lose', 'breaks', 'get back', 'love much']","[""I feel like she's lost to me."", 'he backed me up in my boundaries because he also has those boundaries with her, And is definitely the only reason why she is adhering to those boundaries right now.', 'She won’t adhere to my boundaries.']",No,The topic reflects the narrator's emotional struggles and relationship dynamics rather than providing meaningful insight into the OON's radicalization pathway.
82,81,"['baby', 'pregnant', 'pregnancy', 'first child', 'newborn', 'birth', 'born', '

Topics LLM kept but I did not:


,Topic,Representation,Representative_Docs,Relevance,Relevance_Reason
3,2,"['conspiracy theories', 'theories', 'conspiracies', 'conspiracy theory', 'theorist', 'conspiracy theorist', 'theory', 'theorists', 'conspiracy theorists', 'believe conspiracy']","['I was openminded at first, but the conspiracy theories stopped making sense.', 'So much more - conspiracy theories for everything !', 'About 5 years ago I talked to my mom about some conspiracy theories.']",Yes,"This topic provides insight into the OON's beliefs and engagement with conspiracy theories, which is relevant to understanding their radicalization pathway (Stage 3: After)."
7,6,"['qs', 'adjacent', 'loved ones', 'anyone else', 'related', 'relatives', 'followers', 'family members', 'relative', 'member']","['Consider doing it with your Qs.', 'For those of you who have cut out your Qs, how did you do it?', ""I don't want Qs on here trying to harass us.""]",Yes,"This topic addresses the relationships and dynamics with family members and loved ones who are involved in QAnon, which can provide insight into the OON's post-radicalization impact on their social connections (Stage 3: After)."
8,7,"['family members', 'families', 'family member', 'love family', 'extended family', 'member', 'extended', 'family family', 'want family', 'relatives']","['Has anyone else had to deal with this with a family member?', 'I feel like I’ve lost a family member.', 'Do you have family members like this?']",Yes,"This addresses the impact of radicalization on family dynamics and relationships, which is relevant to understanding the consequences after radicalization (Stage 3: After)."
10,9,"['qmom', 'anon', 'adjacent', 'follower', 'related', 'conspiracies', 'facebook', 'qult', 'theories', 'followers']","['I guess I’m just wondering what personality traits she shares with other Q Anon followers, and/or if anyone has found a way to reach their Q Anon person before they’re too far gone.', 'She kept supporting Trump and Q Anon and I just wish I knew how to get through to her.', ""i know she's into q-anon so she's likely got something from there.""]",Yes,"This topic explores the personality traits and connections of the OON with other QAnon followers, which can provide insights into their beliefs and behaviors after radicalization (Stage 3: After)."
15,14,"['parents parents', 'relationship parents', 'parent', 'like parents', 'lost parents', 'know parents', 'parents want', 'parents also', 'people parents', 'parents really']","['Especially if this were your parent.', ""I'm in my 30's and I shouldn't have this kind of relationship with my parents."", ""My partner feels like I'm prioritizing my parents, my parents want to love me but on their terms, and I just feel like a lost little kid who is stuck with no good options.""]",Yes,"This topic provides insight into the OON's familial relationships and dynamics, which can influence their beliefs and behaviors post-radicalization (Stage 3: After). Understanding the relationship with parents can help explain the emotional and psychological factors that may contribute to the OON's radicalization pathway."
17,16,"['trans', 'gay', 'transgender', 'queer', 'trans people', 'lgbtq', 'lgbt', 'homophobic', 'gender', 'bi']","['mostly because of this shit) trans man.', 'I’m a gay trans man).', 'Most of them are gay and trans.']",Yes,"This topic provides insight into the OON's identity and potential experiences of marginalization, which can contribute to understanding their predispositions and beliefs (Stage 1: Before)."
27,26,"['conspiracy theories', 'conspiracies', 'theories', 'conspiracy theory', 'theory', 'conspiracy theorist', 'theorist', 'conspiracy videos', 'every conspiracy', 'conspiracy theorists']","[""She doesn't seem to be into conspiracy theories but she's still a Right Winger."", 'Her husband is into ""conspiracy theories.""', 'She seems to believe basically all the conspiracy theories.']",Yes,"This provides insight into the OON's beliefs and associations wi

In [55]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

From here I look at the differences between me and GPT and resolve them into one final taxonomy for oon analysis -> save to `data/processed/taxonomy.xlsx`

After certain consideration, we decided that the phase 3 (post-radicalization) has too many categories -> develop sub-categories for those of phase 3